### 读取背景音乐

In [1]:
import wave
import numpy as np

# music = wave.open(r'background.wav', mode='rb')
# music.getparams()
# bg_data = music.readframes(nframes=32191488)
# bg_data = np.frombuffer(bg_data, dtype=np.int16)#音频的格式通常都是16位；buffer缓冲器
# bg_data.shape

# bg_data = np.zeros([44100*15*60], dtype=np.int16)

In [2]:
ding = wave.open(r'息肉音频.wav', mode='rb')
ding.getparams()

_wave_params(nchannels=1, sampwidth=2, framerate=44100, nframes=39710, comptype='NONE', compname='not compressed')

In [3]:
nframes = 39710
nchannels=1
ding_data = ding.readframes(nframes=nframes)
ding_data = np.frombuffer(ding_data, dtype=np.int16)
# ding_data = ding_data.reshape(nframes, 2)
ding_data.shape

(39710,)

### 读取视频息肉出现时间点

In [4]:
import sys
import os
import subprocess
sys.path.append(r'../../NLP/MedicalRecord/Lib')

from MRRecordUtil import load_sheet_arr_dict

In [5]:
# 计算视频秒数
video_fps = 18
def calc_video_len(video_name, base_dir):
    img_folder = os.path.join(os.path.join(base_dir, 'images_crop'), video_name)
    return len(os.listdir(img_folder)) // video_fps

# 合成视频音频
def gen_audio(ding_data, video_secs, framerate, polp_time_arr):
    print(video_secs, framerate, video_secs*framerate)
    print(polp_time_arr)
    bg_data = np.zeros([video_secs*framerate], dtype=np.int16)
    bg_len = bg_data.shape[0]
    start_frames = []
    if ding_data is not None:
        ding_len = ding_data.shape[0]
#         ding_sec = ding_len / framerate
#         print(ding_len)
        print('bg_len: %s, ding_len: %s' % (bg_len, ding_len))
        last_frame = -1
        for ptime in polp_time_arr:
            p_frame = int(ptime * framerate)
            if (last_frame == -1 or p_frame - last_frame > ding_len) and p_frame < bg_len:
                start_frame = p_frame
                end_frame = start_frame + ding_len
                end_frame = end_frame if bg_len > end_frame else bg_len
#                 print(ptime, start_frame, end_frame)
#                 bg_data[start_frame:end_frame, :] = ding_data[0:(end_frame-start_frame), :]
                bg_data[start_frame:end_frame] = ding_data[0:(end_frame-start_frame)]
                last_frame = p_frame
                start_frames.append(start_frame)
            else:
                print(ptime, p_frame - last_frame)
    print(start_frames)
    print(len(polp_time_arr), len(start_frames))
    print([f / framerate for f in start_frames])
    return bg_data

# 写音频
def write_audio(audio_data, secs, framerate=44100, nchannels=1, sampwidth=2):
    os.remove('./generated.wav')
    fp = wave.Wave_write('./generated.wav')
    #设置fp的参数
    fp.setframerate(framerate)
    fp.setnframes(secs*framerate)
    fp.setnchannels(nchannels)
    fp.setsampwidth(sampwidth)

    #写入目录
    fp.writeframes(audio_data.tobytes())
    fp.close()
    
# 读取excel记录视频数据
def get_video_data_from_sheet(sheet_data):
    video_data = []
    for r in sheet_data:
        r_data = {}
        r_data['视频名'] = '%d' % r['视频名']
        r_data['所选参数'] = r['所选参数']
        r_data['息肉出现时间点'] = [float(e) for e in r['息肉出现时间点'].split('\n') if e != '']
        video_data.append(r_data)
    return video_data

def add_audio_for_video_data(v_data, video_folder, base_dir):
    video_secs = calc_video_len(v_data['视频名'], base_dir)
    print('video seconds: %s' % video_secs)
    audo_data = gen_audio(ding_data, video_secs=video_secs, framerate=44100, polp_time_arr=v_data['息肉出现时间点'])
    write_audio(audo_data, video_secs, framerate=44100)
    
    video_path = os.path.join(video_folder, '%s_%s.avi' % (v_data['视频名'], v_data['所选参数']))
    trans_video_path = os.path.join(video_folder, '%s_%s_audio.mp4' % (v_data['视频名'], v_data['所选参数']))
    cmd = 'ffmpeg -i generated.wav -i %s %s' % (video_path, trans_video_path)
    subprocess.call(cmd)

In [7]:
excel = '息肉出现时间点统计.xls'
# r'D:\项目资料\息肉假阳性\202208\有息肉肠镜视频', 
# r'D:\项目资料\息肉假阳性\202208\无息肉肠镜视频'
for base_dir in [r'D:\项目资料\息肉假阳性\202208\无息肉肠镜视频']:
    video_folder = os.path.join(base_dir, 'videos_gen')
    sheet_data = load_sheet_arr_dict(os.path.join(base_dir, excel), 'Sheet1')
    video_data = get_video_data_from_sheet(sheet_data)
    
    for v_data in video_data:
        if len(v_data['息肉出现时间点']) > 0:# and v_data['视频名'] in ['3496']:
            print('processing ', v_data['视频名'])
            add_audio_for_video_data(v_data, video_folder, base_dir)


processing  3419
video seconds: 481
481 44100 21212100
[60.889, 62.5, 63.278, 111.5, 127.778, 300.333, 456.056, 465.167, 475.333]
bg_len: 21212100
63.278 34309
[2685204, 2756250, 4917150, 5635009, 13244685, 20112069, 20513864, 20962185]
9 8
[60.88897959183674, 62.5, 111.5, 127.77798185941043, 300.33299319727894, 456.0559863945578, 465.1669841269841, 475.33299319727894]
processing  3434
video seconds: 320
320 44100 14112000
[6.278, 12.889, 30.167, 210.611, 236.333, 240.944, 241.611, 242.167, 299.833]
bg_len: 14112000
241.611 29415
[276859, 568404, 1330364, 9287945, 10422285, 10625630, 10679564, 13222635]
9 8
[6.27798185941043, 12.888979591836735, 30.166984126984126, 210.6109977324263, 236.3329931972789, 240.94399092970522, 242.16698412698412, 299.83299319727894]
processing  3438
video seconds: 374
374 44100 16493400
[18.167, 43.167, 104.722, 106.0, 108.389, 109.389, 162.056, 162.611, 168.722, 175.5, 176.722, 308.5, 347.833, 349.556, 350.056]
bg_len: 16493400
162.611 24476
350.056 22050


processing  3527
video seconds: 373
373 44100 16449300
[0.056, 23.389, 73.944, 76.222, 94.722, 114.778, 181.056, 208.5, 220.556, 226.167, 229.333, 231.333, 234.667, 238.667, 242.056, 244.056, 245.556, 248.944, 250.556, 262.611, 264.444, 273.389, 274.444, 276.722, 278.944, 287.389, 288.111, 312.722, 316.722, 320.444, 323.222, 340.444, 341.889, 348.722, 349.722, 352.556, 356.722, 362.111, 367.0, 369.556, 372.833, 373.444]
bg_len: 16449300
288.111 31841
373.444 26945
[2469, 1031454, 3260930, 3361390, 4177240, 5061709, 7984569, 9194850, 9726519, 9973964, 10113585, 10201785, 10348814, 10525214, 10674669, 10762869, 10829019, 10978430, 11049519, 11581145, 11661980, 12056454, 12102980, 12203440, 12301430, 12673854, 13791040, 13967440, 14131580, 14254090, 15013580, 15077304, 15378640, 15422740, 15547719, 15731440, 15969095, 16184700, 16297419, 16441935]
42 40
[0.05598639455782313, 23.388979591836733, 73.94399092970522, 76.22199546485261, 94.72199546485261, 114.77798185941043, 181.05598639455783

processing  3530
video seconds: 365
365 44100 16096500
[5.056, 11.0, 14.167, 16.944, 19.444, 23.722, 27.222, 28.333, 29.889, 52.667, 53.444, 69.667, 78.5, 103.722, 105.611, 110.611, 114.167, 115.667, 117.611, 128.0, 128.944, 135.611, 137.056, 138.722, 139.556, 140.167, 140.889, 143.111, 144.5, 146.333, 147.722, 154.389, 157.389, 159.778, 171.722, 192.167, 195.111, 200.722, 202.056, 202.556, 204.944, 205.667, 215.556, 218.0, 221.056, 228.778, 231.944, 234.667, 236.556, 254.222, 257.944, 262.278, 263.389, 264.444, 267.0, 268.0, 269.833, 276.611, 281.222, 283.778, 285.278, 287.889, 290.222, 293.833, 294.833, 309.167, 310.111, 315.444, 324.889, 325.333, 327.611, 330.278, 334.889, 336.222, 339.0, 341.833, 346.5, 347.778, 349.556, 359.167, 360.833, 361.556, 362.944, 363.389, 364.278]
bg_len: 16096500
53.444 34266
139.556 36779
140.889 31840
202.556 22050
205.667 31884
325.333 19581
361.556 31884
363.389 19624
[222969, 485100, 624764, 747230, 857480, 1046140, 1200490, 1249485, 1318104, 232261

processing  3507
video seconds: 440
440 44100 19404000
[0.167, 2.944, 6.944, 8.5, 12.389, 13.833, 15.833, 21.556, 30.556, 33.389, 34.5, 35.722, 46.778, 51.778, 54.667, 59.722, 60.611, 66.111, 71.889, 73.056, 75.722, 80.278, 82.0, 84.333, 85.667, 87.389, 88.278, 88.889, 90.222, 92.722, 94.889, 106.611, 115.278, 131.944, 140.722, 142.444, 149.5, 150.389, 155.056, 169.111, 198.5, 219.944, 248.667, 251.389, 260.444, 263.0, 279.833, 284.722, 288.111, 296.722, 321.0, 322.056, 324.556, 326.556, 327.611, 331.222, 339.778, 340.222, 343.444, 364.056, 369.333, 383.111, 384.667, 388.389, 388.944, 393.111, 396.722, 398.889, 406.556, 417.5, 421.667, 424.167, 426.889, 433.667, 437.0]
bg_len: 19404000
60.611 39205
88.278 39205
150.389 39204
340.222 19581
388.944 24476
[7364, 129830, 306230, 374850, 546354, 610035, 698235, 950619, 1347519, 1472454, 1521450, 1575340, 2062909, 2283409, 2410814, 2633740, 2915495, 3170304, 3221769, 3339340, 3540259, 3616200, 3719085, 3777914, 3853854, 3920004, 3978790, 408

processing  3466
video seconds: 382
382 44100 16846200
[0.056, 2.389, 3.5, 5.222, 5.778, 8.0, 9.833, 11.611, 17.833, 22.556, 24.5, 25.0, 27.333, 28.167, 29.0, 30.667, 34.722, 37.833, 38.778, 39.833, 46.056, 47.444, 49.722, 53.889, 57.722, 60.556, 62.556, 67.222, 70.0, 71.722, 74.167, 77.5, 78.333, 79.389, 83.889, 84.944, 86.0, 89.0, 89.667, 90.778, 96.278, 100.611, 101.556, 102.444, 106.0, 108.056, 108.833, 112.389, 119.222, 119.722, 122.722, 125.167, 126.611, 147.556, 154.333, 163.167, 164.0, 165.111, 166.0, 168.944, 174.667, 177.833, 179.833, 181.444, 183.944, 189.222, 190.778, 193.833, 194.667, 196.111, 202.333, 206.222, 207.111, 213.278, 214.556, 218.167, 223.944, 230.389, 232.222, 234.444, 240.556, 245.889, 247.667, 248.444, 253.333, 255.611, 261.444, 263.667, 264.833, 269.944, 271.0, 273.111, 275.778, 277.056, 277.556, 279.111, 280.889, 282.111, 283.389, 287.944, 295.167, 299.778, 302.389, 304.111, 307.167, 308.722, 309.722, 310.944, 313.667, 335.056, 336.833, 349.444, 350.0, 353

processing  3464
video seconds: 364
364 44100 16052400
[0.667, 1.222, 1.833, 4.556, 8.389, 9.0, 10.389, 11.222, 15.278, 26.0, 27.222, 29.167, 31.611, 33.889, 36.667, 41.111, 41.889, 50.389, 53.5, 55.722, 56.278, 58.056, 59.056, 60.0, 62.667, 63.722, 70.722, 71.222, 71.944, 77.667, 80.722, 82.5, 83.167, 84.5, 85.278, 86.111, 88.944, 95.333, 98.667, 99.556, 103.333, 110.222, 111.611, 115.444, 118.278, 120.056, 121.056, 126.722, 129.167, 130.111, 132.0, 134.5, 134.944, 136.056, 138.111, 140.944, 145.5, 146.667, 150.167, 158.611, 159.333, 183.611, 184.222, 188.556, 191.444, 203.833, 207.722, 220.889, 222.111, 223.167, 224.556, 228.222, 233.444, 234.444, 238.111, 240.0, 242.722, 244.0, 247.278, 251.389, 252.167, 260.556, 263.0, 268.444, 271.667, 284.167, 284.944, 290.722, 293.222, 295.889, 297.0, 301.167, 312.167, 313.722, 314.444, 315.222, 316.222, 336.667, 339.889, 340.889, 344.278, 349.333, 351.333, 359.556, 360.722, 361.778, 362.556]
bg_len: 16052400
1.222 24476
9.0 26946
11.222 36736
4

processing  3475
video seconds: 398
398 44100 17551800
[0.333, 1.722, 3.389, 6.056, 8.889, 10.0, 11.167, 13.556, 15.056, 15.944, 22.5, 32.5, 33.056, 36.167, 38.444, 40.444, 41.833, 42.278, 44.333, 51.5, 52.444, 53.722, 55.667, 56.778, 57.833, 59.444, 60.556, 64.389, 64.944, 66.333, 69.333, 70.667, 74.5, 75.111, 76.0, 76.444, 77.722, 80.5, 86.111, 90.611, 93.722, 97.111, 101.611, 108.278, 109.333, 113.889, 128.167, 129.944, 149.444, 155.667, 156.778, 158.056, 158.611, 159.722, 162.444, 163.889, 166.111, 167.944, 170.444, 171.667, 172.667, 174.556, 176.556, 179.111, 180.611, 181.611, 183.111, 199.778, 200.889, 208.556, 211.833, 214.778, 216.333, 217.778, 225.389, 227.611, 229.278, 235.389, 236.556, 239.0, 240.889, 241.5, 242.167, 243.111, 244.667, 246.056, 246.611, 249.667, 255.5, 261.0, 262.333, 263.444, 268.167, 269.167, 271.444, 272.778, 273.5, 275.944, 281.056, 283.333, 284.833, 287.167, 290.833, 291.778, 299.333, 303.278, 304.389, 306.333, 313.0, 314.5, 315.056, 316.889, 317.5, 320.

In [11]:
704 * 44100

31046400